In [ ]:
# install (usually not needed) and imports
!pip install -q category_encoders

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
# If you placed the CSV inside your Drive (My Drive), set the path below.
# Example paths you might try:
candidates = [
    "/content/drive/My Drive/loan_data.csv",
    "/content/drive/My Drive/LoanApproval.csv",
    "/content/drive/My Drive/loan.csv",
    "/content/drive/My Drive/Loan_Status.csv"
]

df = None
for p in candidates:
    if os.path.exists(p):
        print("Found file:", p)
        df = pd.read_csv(p)
        break

if df is None:
    print("No candidate file found at default paths. You'll be prompted to upload a file in the next cell.")
else:
    print("Loaded dataframe with shape:", df.shape)
    display(df.head())
from google.colab import files
import pandas as pd

print("Upload your CSV file 👇")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print("Loaded:", filename)
display(df.head())
print("Shape:", df.shape)
print("\nColumn types:\n", df.dtypes)
print("\nMissing values:\n", df.isnull().sum())

df.head()
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Make a copy
data = df.copy()

# ---- 1. DROP Loan_ID ----
data.drop(columns=['Loan_ID'], inplace=True)

# ---- 2. HANDLE MISSING VALUES ----
# Fill numeric columns
data['LoanAmount'].fillna(data['LoanAmount'].median(), inplace=True)
data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].median(), inplace=True)
data['Credit_History'].fillna(data['Credit_History'].median(), inplace=True)

# Fill categorical columns
for col in ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']:
    data[col].fillna(data[col].mode()[0], inplace=True)

# ---- 3. ENCODE CATEGORICALS ----
le = LabelEncoder()
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = le.fit_transform(data[col])

# ---- 4. SPLIT X and y ----
X = data.drop('Loan_Status', axis=1)
y = data['Loan_Status']  # Y/N encoded automatically by LabelEncoder above

# ---- 5. TRAIN TEST SPLIT ----
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Dataset ready! 🎉")
print("X shape:", X.shape)
print("y shape:", y.shape)
X.head()
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
